In [1]:
from itertools import combinations
from phevaluator import evaluate_cards, evaluate_omaha_cards
import random
#from concurrent.futures import ProcessPoolExecutor, as_completed

In [2]:
# Because the unconfigured hands tuple NLHE hand elements must be configured in a specific way (Strong and Weak)
# We need this dictionary to rank and compare NLHE hands, swapping element locations in the tuple when necessary. 

absolute_hand_strength = {
    "AA": 0, "KK": 1, "QQ": 2, "JJ": 3, "TT": 4, "99": 5, "88": 6, "77": 7, "66": 8, "55": 9, "44": 10, "33": 11,"22": 12, 

    "AK": 13, "AQ": 14, "AJ": 15, "AT": 16, "A9": 17, "A8": 18, "A7": 19, "A6": 20, "A5": 21, "A4": 22, "A3": 23, "A2": 24, 
    "KA": 13, "QA": 14, "JA": 15, "TA": 16, "9A": 17, "8A": 18, "7A": 19, "6A": 20, "5A": 21, "4A": 22, "3A": 23, "2A": 24, 

    "KQ": 25, "KJ": 26, "KT": 27, "K9": 28, "K8": 29, "K7": 30, "K6": 31, "K5": 32, "K4": 33, "K3": 34, "K2": 35,
    "QK": 25, "JK": 26, "TK": 27, "9K": 28, "8K": 29, "7K": 30, "6K": 31, "5K": 32, "4K": 33, "3K": 34, "2K": 35,

    "QJ": 36, "QT": 37, "Q9": 38, "Q8": 39, "Q7": 40, "Q6": 41, "Q5": 42, "Q4": 43, "Q3": 44, "Q2": 45, 
    "JQ": 36, "TQ": 37, "9Q": 38, "8Q": 39, "7Q": 40, "6Q": 41, "5Q": 42, "4Q": 43, "3Q": 44, "2Q": 45, 

    "JT": 46, "J9": 47, "J8": 48, "J7": 49, "J6": 50, "J5": 51, "J4": 52, "J3": 53, "J2": 54,
    "TJ": 46, "9J": 47, "8J": 48, "7J": 49, "6J": 50, "5J": 51, "4J": 52, "3J": 53, "2J": 54,

    "T9": 55, "T8": 56, "T7": 57, "T6": 58, "T5": 59, "T4": 60, "T3": 61, "T2": 62,
    "9T": 55, "8T": 56, "7T": 57, "6T": 58, "5T": 59, "4T": 60, "3T": 61, "2T": 62,

    "98": 63, "97": 64, "96": 65, "95": 66, "94": 67, "93": 68, "92": 69,
    "89": 63, "79": 64, "69": 65, "59": 66, "49": 67, "39": 68, "29": 69,

    "87": 70, "86": 71, "85": 72, "84": 73, "83": 74, "82": 75,
    "78": 70, "68": 71, "58": 72, "48": 73, "38": 74, "28": 75,

    "76": 76, "75": 77, "74": 78, "73": 79, "72": 80,
    "67": 76, "57": 77, "47": 78, "37": 79, "27": 80,

    "65": 81, "64": 82, "63": 83, "62": 84,
    "56": 81, "46": 82, "36": 83, "26": 84,

    "54": 85, "53": 86, "52": 87,
    "45": 85, "35": 86, "25": 87,

    "43": 88, "42": 89,
    "34": 88, "24": 89,

    "32": 90,
    "23": 90
}

In [3]:
# Takes in 12 unconfigured cards and NLHE absolute hand strength dictionary
# returns exhaustive 207,900 configured combos from the 4 hand categories
def generate_all_configurable_hands(unconfigured_cards, absolute_hand_strength):
    all_hands = []

    # Exhaustively generate (12 choose 4) combos for 1st {PLO} hand. returns remaining_after_PLO_HAND = 8 cards left
    for PLO_HAND in combinations(unconfigured_cards, 4):
        remaining_after_PLO_HAND = [card for card in unconfigured_cards if card not in PLO_HAND]
        
        # Exhaustively generate (8 choose 4) combos for 2nd {PLO} hand from remaining cards. returns remaining_after_PLO_HAND_2 = 4 cards left
        for PLO_HAND_2 in combinations(remaining_after_PLO_HAND, 4):
            remaining_after_PLO_HAND_2 = [card for card in remaining_after_PLO_HAND if card not in PLO_HAND_2]

            # Exhaustively generate (4 choose 2) combos for 1st {NLHE} hand and 2nd {NLHE} hand from remaining cards.
            # The last 2 cards automatically make the 2nd {NLHE} hand.
            for NLHE_HAND_1 in combinations(remaining_after_PLO_HAND_2, 2):
                remaining_after_NLHE_HAND_1 = [card for card in remaining_after_PLO_HAND_2 if card not in NLHE_HAND_1]
                NLHE_HAND_2 = tuple(remaining_after_NLHE_HAND_1)

                # Removes suit from NLHE hand cards so we can calculate and compare hand strength for tuple element placement.
                NLHE_HAND_1_RAW = ''.join([card[0] for card in NLHE_HAND_1])
                NLHE_HAND_2_RAW = ''.join([card[0] for card in NLHE_HAND_2])

                # Compare and swap NLHE hands element location in tuple based on strength if needed. NOTE: Larger rank = worse hand.
                # In the case that two NLHE hands are identical, they can remain in same location because player arbitrarily chooses anyway.
                if absolute_hand_strength.get(NLHE_HAND_1_RAW, 0) > absolute_hand_strength.get(NLHE_HAND_2_RAW, 0):
                    NLHE_HAND_1, NLHE_HAND_2 = NLHE_HAND_2, NLHE_HAND_1

                # Tuple elements MUST be output like this: ((PLO_1), (PLO_2), (NLHE_STRONGER), (NLHE_WEAKER))
                all_hands.append((PLO_HAND, PLO_HAND_2, NLHE_HAND_1, NLHE_HAND_2))

    return all_hands

def generate_remaining_deck(player_1_unconfigured_hand, player_2_unconfigured_hand):
    all_cards = [f"{rank}{suit}" for suit in 'shdc' for rank in '23456789TJQKA']
    excluded_cards = set(player_1_unconfigured_hand) | set(player_2_unconfigured_hand)
    remaining_deck = [card for card in all_cards if card not in excluded_cards]
    return remaining_deck

def generate_all_boards(player_1_unconfigured_hand, player_2_unconfigured_hand):
    remaining_deck = generate_remaining_deck(player_1_unconfigured_hand, player_2_unconfigured_hand)
    return list(combinations(remaining_deck, 5))

In [4]:
# Example User Inputs
player_1_unconfigured_hand = ("3d", "9d", "3c", "6c", "6h", "Ad", "Qh", "3s", "2c", "As", "Td", "9c")
player_2_unconfigured_hand = ("7h", "7s", "Qd", "Jc", "Ac", "2h", "4c", "4d", "8s", "Kh", "9s", "5d")

In [5]:
# Outputs
player_1_configured_hands = generate_all_configurable_hands(player_1_unconfigured_hand, absolute_hand_strength)
player_2_configured_hands = generate_all_configurable_hands(player_2_unconfigured_hand, absolute_hand_strength)
all_boards = generate_all_boards(player_1_unconfigured_hand, player_2_unconfigured_hand)

#random.seed(42)
player_1_random_hands = random.sample(player_1_configured_hands, 2000)
player_2_random_hands = random.sample(player_2_configured_hands, 2000)
random_boards = random.sample(all_boards, 3000) 

print(f"All {len(player_1_configured_hands)} hand configuration combos exhaustively saved.")
print(f"All {len(all_boards)} boards exhaustively saved.")
print(f"Picked {len(player_1_random_hands)} Player 1 hands & {len(player_2_random_hands)} Player 2 hands.")
print(f"Picked {len(random_boards)} random boards.")
print(f"Player 1 Sample: {player_1_random_hands[0]}\nPlayer 2 Sample: {player_2_random_hands[0]}")
print(f"Random Board 1: {random_boards[0]}, Random Board 2: {random_boards[1]}")

All 207900 hand configuration combos exhaustively saved.
All 98280 boards exhaustively saved.
Picked 2000 Player 1 hands & 2000 Player 2 hands.
Picked 3000 random boards.
Player 1 Sample: (('3d', '3s', '2c', 'Td'), ('6c', 'Qh', 'As', '9c'), ('9d', 'Ad'), ('3c', '6h'))
Player 2 Sample: (('7h', 'Qd', 'Jc', '4c'), ('7s', '2h', '4d', '9s'), ('Ac', '5d'), ('8s', 'Kh'))
Random Board 1: ('Js', '3h', '4h', '2d', '7d'), Random Board 2: ('5s', 'Ks', '3h', 'Th', '6d')


In [6]:
# Example Values
player_1_configured_hand_example = (('3d', '3c', 'Ad', '2c'), ('9d', '6h', 'Qh', 'As'), ('3s', 'Td'), ('6c', '9c'))
player_2_configured_hand_example = (('7h', '7s', '4c', 'Kh'), ('Ac', '2h', '8s', '9s'), ('Qd', '5d'), ('Jc', '4d'))
board_example = ('5s', '3h', '8h', 'Ah', '5c')

# Determines the ranking of the hand made by each player for each category. Then compares the rankings and determines winner for each category.
# NOTE: Larger Rank = Worse Hand. Smaller Rank = Better Hand.
# NOTE: Handles chop edge cases for NLHE and PLO, and handles the double points awarded edgecase.
# Calculates net points, which is the money won/lost for the player. (1 point = $1 for example).
def compute_net_points(player_1_configured_hand, player_2_configured_hand, board):
    PLAYER_1_TOTAL_POINTS = 0

    # PLO Hand 1 Evaluation. (4 POINTS). 
    PLAYER_1_PLO_HAND_1_RANK = evaluate_omaha_cards(*(board + player_1_configured_hand[0]))
    PLAYER_2_PLO_HAND_1_RANK = evaluate_omaha_cards(*(board + player_2_configured_hand[0]))
    if PLAYER_1_PLO_HAND_1_RANK < PLAYER_2_PLO_HAND_1_RANK:
        PLAYER_1_TOTAL_POINTS += 4
    elif PLAYER_1_PLO_HAND_1_RANK == PLAYER_2_PLO_HAND_1_RANK:
        PLAYER_1_TOTAL_POINTS += 2

    # PLO Hand 2 Evaluation. (3 POINTS)
    PLAYER_1_PLO_HAND_2_RANK = evaluate_omaha_cards(*(board + player_1_configured_hand[1]))
    PLAYER_2_PLO_HAND_2_RANK = evaluate_omaha_cards(*(board + player_2_configured_hand[1]))
    if PLAYER_1_PLO_HAND_2_RANK < PLAYER_2_PLO_HAND_2_RANK:
        PLAYER_1_TOTAL_POINTS += 3
    elif PLAYER_1_PLO_HAND_2_RANK == PLAYER_2_PLO_HAND_2_RANK:
        PLAYER_1_TOTAL_POINTS += 1.5
    
    # NLHE Stronger Hand Evaluation. (2 POINTS)
    PLAYER_1_NLHE_HAND_STRONGER_RANK = evaluate_cards(*(board + player_1_configured_hand[2]))
    PLAYER_2_NLHE_HAND_STRONGER_RANK = evaluate_cards(*(board + player_2_configured_hand[2]))
    if PLAYER_1_NLHE_HAND_STRONGER_RANK < PLAYER_2_NLHE_HAND_STRONGER_RANK:
        PLAYER_1_TOTAL_POINTS += 2
    elif PLAYER_1_NLHE_HAND_STRONGER_RANK == PLAYER_2_NLHE_HAND_STRONGER_RANK:
        PLAYER_1_TOTAL_POINTS += 1

    # NLHE Weaker Hand Evaluation. (1 POINT)
    PLAYER_1_NLHE_HAND_WEAKER_RANK = evaluate_cards(*(board + player_1_configured_hand[3]))
    PLAYER_2_NLHE_HAND_WEAKER_RANK = evaluate_cards(*(board + player_2_configured_hand[3]))
    if PLAYER_1_NLHE_HAND_WEAKER_RANK < PLAYER_2_NLHE_HAND_WEAKER_RANK:
        PLAYER_1_TOTAL_POINTS += 1
    elif PLAYER_1_NLHE_HAND_WEAKER_RANK == PLAYER_2_NLHE_HAND_WEAKER_RANK:
        PLAYER_1_TOTAL_POINTS += 0.5
    
    # There is an edgecase in the game. If a player wins all categories, they earn double points.
    if PLAYER_1_TOTAL_POINTS == 10:
        PLAYER_1_NET_POINTS = 20
    elif PLAYER_1_TOTAL_POINTS == 0:
        PLAYER_1_NET_POINTS = -20 
    else:
        PLAYER_1_NET_POINTS = PLAYER_1_TOTAL_POINTS - (10 - PLAYER_1_TOTAL_POINTS)

    PLAYER_2_NET_POINTS = -PLAYER_1_NET_POINTS 

    return PLAYER_1_NET_POINTS
    #print("--------------------------------------------")
    #print(f"Player 1 PLO Hand 1: {PLAYER_1_PLO_HAND_1_RANK}, Player 2 PLO Hand 1: {PLAYER_2_PLO_HAND_1_RANK}")
    #print(f"Player 1 PLO Hand 2: {PLAYER_1_PLO_HAND_2_RANK}, Player 2 PLO Hand 2: {PLAYER_2_PLO_HAND_2_RANK}")
    #print(f"Player 1 NLHE Stronger: {PLAYER_1_NLHE_HAND_STRONGER_RANK}, Player 2 NLHE Stronger: {PLAYER_2_NLHE_HAND_STRONGER_RANK}")
    #print(f"Player 1 NLHE Weaker: {PLAYER_1_NLHE_HAND_WEAKER_RANK}, Player 2 NLHE Weaker: {PLAYER_2_NLHE_HAND_WEAKER_RANK}")
    #print(f"Player 1 raw points: {PLAYER_1_TOTAL_POINTS} out of 10")
    #print(f"Player 1 net points: {PLAYER_1_NET_POINTS}")
    #print(f"Player 2 net points: {PLAYER_2_NET_POINTS}")
    #print("--------------------------------------------")
compute_net_points(player_1_configured_hand_example, player_2_configured_hand_example, board_example)

4

In [7]:
def compute_all_matchups_sequential(player_1_hands, player_2_hands, boards):
    results = []
    for board in boards:
        for p1_hand in player_1_hands:
            for p2_hand in player_2_hands:
                result = compute_net_points(p1_hand, p2_hand, board)  # Adjusted order to match function signature
                results.append(result)
    return results

# NOTE: you'll blow up ur cpu if u put this at like (100,100,100)
test_results = compute_all_matchups_sequential(player_1_random_hands[:50], player_2_random_hands[:50], random_boards[:50])

In [8]:
def compute_average(test_results):
    total = sum(test_results)
    average = total / len(test_results)
    return average

average = compute_average(test_results)
print("Average:", average)

Average: -1.957312


In [ ]:
"""
NOTE: This was the `generate_all_configurable_hands` when it generated on the fly rather than append all 207,900 combos into a list.

# Takes in 12 unconfigured cards and strength dictionary and yields all 207,900 comobos for the necessary categories.
def generate_all_configurable_hands(unconfigured_cards, absolute_hand_strength):

    # Exhaustively generate (12 choose 4) combos for 1st {PLO} hand. returns remaining_after_PLO_HAND = 8 cards left
    for PLO_HAND in combinations(unconfigured_cards, 4):
        remaining_after_PLO_HAND = [card for card in unconfigured_cards if card not in PLO_HAND]

        # Exhaustively generate (8 choose 4) combos for 2nd {PLO} hand from remaining cards. returns remaining_after_PLO_HAND_2 = 4 cards left
        for PLO_HAND_2 in combinations(remaining_after_PLO_HAND, 4):
            remaining_after_PLO_HAND_2 = [card for card in remaining_after_PLO_HAND if card not in PLO_HAND_2]
            
            # Exhaustively generate (4 choose 2) combos for 1st {NLHE} hand and 2nd {NLHE} hand from remaining cards
            for NLHE_HAND_1 in combinations(remaining_after_PLO_HAND_2, 2):
                remaining_after_NLHE_HAND_1 = [card for card in remaining_after_PLO_HAND_2 if card not in NLHE_HAND_1] # returns the 2 cards left
                NLHE_HAND_2 = tuple(remaining_after_NLHE_HAND_1) # Last 2 cards automatically make 2nd {NLHE} hand. Always just (2 choose 2) = 1.
                
                # Convert NLHE hands to string representations
                NLHE_HAND_1_STR = ''.join(([card[0] for card in NLHE_HAND_1]))
                NLHE_HAND_2_STR = ''.join(([card[0] for card in NLHE_HAND_2]))

                # Compare and potentially swap NLHE hands based on strength. NOTE: Larger rank = worse hand.
                # In the case that two NLHE hands are identical, they can remain in same location because player arbitrarily chooses anyway.
                if absolute_hand_strength[NLHE_HAND_1_STR] > absolute_hand_strength[NLHE_HAND_2_STR]:
                    NLHE_HAND_1, NLHE_HAND_2 = NLHE_HAND_2, NLHE_HAND_1
                
                # Tuple elements must be output like this: ((PLO_1), (PLO_2), (NLHE_STRONGER), (NLHE_WEAKER))
                yield PLO_HAND, PLO_HAND_2, NLHE_HAND_1, NLHE_HAND_2


total_combos = sum(1 for _ in generate_all_configurable_hands(example_unconfigured_cards, absolute_hand_strength))
print(f"Total combinations: {total_combos}")
count = 0 
for combo in generate_all_configurable_hands(example_unconfigured_cards, absolute_hand_strength):
    print(combo)
    count += 1
    if count >= 2:
        break
"""